In [1]:
#import needed package
#---------------------------------------------------------
import pandas as pd
import numpy as np
import sklearn
import autosklearn
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
import datetime
import autosklearn.classification
from sklearn.preprocessing import LabelEncoder #OrdinalEncoder

/anaconda/envs/py35/lib/python3.5/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
#print versions
#---------------------------------------------------------
print('The scikit-learn version is {}.'.format(sklearn.__version__))
print('The Auto scikit-learn version is {}.'.format(autosklearn.__version__))

The scikit-learn version is 0.19.2.
The Auto scikit-learn version is 0.5.2.


In [3]:
#Load Data from CSV
#--------------------------------------------------------
filepath = '/data/home/eissa/Downloads/datasets_88-BayesianNetworkGenerator_hypothyroid.csv'
rawdata = pd.read_csv( filepath ,  sep=','  ,  low_memory=False )
rawdata.shape

(1000000, 30)

In [4]:
rawdata.head(10)

,age,sex,on_thyroxine,query_on_thyroxine,on_antithyroid_medication,sick,pregnant,thyroid_surgery,I131_treatment,query_hypothyroid,...,TT4_measured,TT4,T4U_measured,T4U,FTI_measured,FTI,TBG_measured,TBG,referral_source,Class
0,'\'B1of3\'',F,f,f,f,t,f,f,f,f,...,t,'\'B1of3\'',t,'\'B2of3\'',t,'\'B1of3\'',f,'\'All\'',other,negative
1,'\'B1of3\'',F,f,f,f,t,f,f,f,f,...,t,'\'B1of3\'',t,'\'B2of3\'',t,'\'B1of3\'',f,'\'All\'',SVI,negative
2,'\'B1of3\'',M,t,f,f,f,f,f,f,f,...,t,'\'B1of3\'',t,'\'B2of3\'',t,'\'B1of3\'',f,'\'All\'',other,negative
3,'\'B1of3\'',F,f,f,f,f,f,f,f,f,...,t,'\'B1of3\'',t,'\'B1of3\'',t,'\'B2of3\'',f,'\'All\'',SVI,negative
4,'\'B1of3\'',F,f,f,f,f,f,f,f,f,...,t,'\'B1of3\'',t,'\'B1of3\'',t,'\'B1of3\'',f,'\'All\'',SVI,negative
5,'\'B1of3\'',F,f,f,f,f,f,f,f,f,...,t,'\'B1of3\'',t,'\'B2of3\'',t,'\'B1of3\'',f,'\'All\'',SVHD,negative
6,'\'B1of3\'',F,f,f,f,f,f,f,f,f,...,t,'\'B1of3\'',t,'\'B2of3\'',t,'\'B1of3\'',f,'\'All\'',other,negative
7,'\'B1of3\'',F,f,f,f,f,f,f,f,f,...,t,'\'B1of3\'',t,'\'B2of3\'',t,'\'B1of3\'',f,'\'All\'',other,negative
8,'\'B1of3\'',F,f,f,f,f,f,f,f,f,...,t,'\'B1of3\'',t,'\'B2of3\'',t,'\'B1of3\'',f,'\'All\'',SVI,negative
9,'\'B1of3\'',M,f,f,f,f,f,f,f,f,...,t,'\'B1of3\'',t,'\'B1of3\'',t,'\'B1of3\'',f,'\'All\'',other,compensated_hypothyroid


In [5]:
#Categorical feature encoding
#--------------------------------------------------------
label = 'Class'
le = LabelEncoder()
for column in rawdata.columns: 
    rawdata[column] = le.fit_transform(rawdata[column])

In [ ]:
rawdata.head(10)

In [6]:
# split data into Testing and training
# ---------------------------------------------------------------------------------------------------------------------
np.random.seed(1234)
msk = np.random.rand(len(rawdata)) < 0.8
training_data = rawdata[msk]
Testing_data = rawdata[~msk]

X_train = training_data.loc[:, training_data.columns != label]
Y_train = training_data.loc[:, training_data.columns == label]

X_test = Testing_data.loc[:, Testing_data.columns != label]
Y_test = Testing_data.loc[:, Testing_data.columns == label]

In [11]:
# auto ML
# ---------------------------------------------------------------------------------------------------------------------
t1 = datetime.datetime.now()
automl = autosklearn.classification.AutoSklearnClassifier(time_left_for_this_task=100, 
                                                          ensemble_size= 1,
                                                          #include_preprocessors=["no_preprocessing"],
                                                          ensemble_memory_limit=45000, 
                                                          seed=1, 
                                                          ml_memory_limit=45000)
automl.fit(X_train, Y_train)
y_hat = automl.predict(X_test)
t2 = datetime.datetime.now()
print("Time:" + str((t2 - t1).total_seconds()))
print("Accuracy score", accuracy_score(Y_test, y_hat))


/anaconda/envs/py35/lib/python3.5/site-packages/autosklearn/automl.py:887: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Will change shape via np.ravel().
  y = self._check_y(y)
/anaconda/envs/py35/lib/python3.5/site-packages/autosklearn/evaluation/train_evaluator.py:197: RuntimeWarning: Mean of empty slice
  Y_train_pred = np.nanmean(Y_train_pred_full, axis=0)


[WARNING] [2020-01-06 12:18:07,167:EnsembleBuilder(1):81150a0c2b16e8f2e24bfce3e3659d07] No models better than random - using Dummy Score!
[WARNING] [2020-01-06 12:18:07,233:EnsembleBuilder(1):81150a0c2b16e8f2e24bfce3e3659d07] No models better than random - using Dummy Score!
Time limit for a single run is higher than total time limit. Capping the limit for a single run to the total time given to SMAC (97.174727)
[WARNING] [2020-01-06 12:18:09,242:EnsembleBuilder(1):81150a0c2b16e8f2e24bfce3e3659d07] No models better than random - using Dummy Score!
[WARNING] [2020-01-06 12:18:11,252:EnsembleBuilder(1):81150a0c2b16e8f2e24bfce3e3659d07] No models better than random - using Dummy Score!
[WARNING] [2020-01-06 12:18:13,263:EnsembleBuilder(1):81150a0c2b16e8f2e24bfce3e3659d07] No models better than random - using Dummy Score!
[WARNING] [2020-01-06 12:18:15,273:EnsembleBuilder(1):81150a0c2b16e8f2e24bfce3e3659d07] No models better than random - using Dummy Score!
[WARNING] [2020-01-06 12:18:17,2

In [ ]:
automl.cv_results_

In [ ]:
automl.show_models(

In [ ]:
automl.sprint_statistics()

In [ ]:
print("Accuracy score", accuracy_score(Y_test, y_hat))